In [2]:
import pyspark.sql.functions
from pyspark.pandas.internal import InternalField
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("cleaning").getOrCreate()

In [3]:
# Name of the Spark application instance
app_name = spark.conf.get("spark.app.name")

# Driver TCP port
driver_tcp_port = spark.conf.get("spark.driver.port")

# Number of join partitions
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

# Show the results
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions)

Name: cleaning
Driver TCP port: 58844
Number of partitions: 200


In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdulwadood11220/usa-house-sales-data")

print("Path to dataset files:", path)
fileName_path = path + "\\us_house_Sales_data.csv"
print(fileName_path)

C:\Users\kizer\miniconda3\envs\spark354_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\kizer\.cache\kagglehub\datasets\abdulwadood11220\usa-house-sales-data\versions\1
C:\Users\kizer\.cache\kagglehub\datasets\abdulwadood11220\usa-house-sales-data\versions\1\us_house_Sales_data.csv


In [6]:
from pyspark.sql.types import *
fileSchema= StructType([
    StructField("Price", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Zipcode", IntegerType(), True),
    StructField("State", StringType(), True),
    StructField("Bedrooms", StringType(), True), #need to be changed to integer and remove bds from data
    StructField("Bathrooms", StringType(), True), #need to be changed to integer and remove ba from data
    StructField("Area (Sqft)", StringType(), True), # need to be changed to integer and remove sqft in data
    StructField("Lot Size", StringType(), True), # need to be changed to integer and remove sqft in data
    StructField("Year Built", IntegerType(), True),
    StructField("Days on Market", IntegerType(), True),
    StructField("Property Type", StringType(), True),
    StructField("MLS ID", StringType(), True),
    StructField("Listing Agent", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Listing URL", StringType(), True),  
])
df = spark.read.format('csv').option("header", "true").load(fileName_path, schema=fileSchema)

In [7]:
df.show()

+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+
|     Price|             Address|         City|Zipcode|State|Bedrooms|Bathrooms|Area (Sqft)| Lot Size|Year Built|Days on Market|Property Type| MLS ID|       Listing Agent|  Status|         Listing URL|
+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+
|  $554,217|5926 Oak Ave, San...|    San Diego|  65383|   CA|   1 bds|     3 ba|   772 sqft|4757 sqft|      1959|           101|    Townhouse|Z104635|Alex Johnson - Co...|For Sale|https://www.zillo...|
|  $164,454|9583 Oak Ave, Fre...|       Fresno|  79339|   IL|   1 bds|     1 ba|  2348 sqft|3615 sqft|      1969|            46|    Apartment|Z535721|Emily Davis - Cen...|    Sold|https://www.

In [8]:
# Store the number of partitions in variable
before = df.rdd.getNumPartitions()

# Configure Spark to use 500 partitions
spark.conf.set('spark.sql.shuffle.partitions', 500)

# Recreate the DataFrame using the departures data file
df = spark.read.format('csv').option("header", "true").load(fileName_path, schema=fileSchema)

# Print the number of partitions for each instance
print("Partition count before change: %d" %before)
print("Partition count after change: %d" % df.rdd.getNumPartitions())

Partition count before change: 1
Partition count after change: 1


In [8]:
#parquet 
#loading:
# df = spark.rea.format('parquet').load(fileName_path).loa('filename.parquet')
# df = spark.read.parquet('filename.parquet')
#saving:
# df.write.format('parquet').save('filename.parquet')
# df.write.parquet('filename.parquet')

#sql:
# dataframe -> tempView -> sql

In [9]:
df_sold = df.filter( df.Status == 'Sold')
df_sold.show()

+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+------+--------------------+
|     Price|             Address|         City|Zipcode|State|Bedrooms|Bathrooms|Area (Sqft)| Lot Size|Year Built|Days on Market|Property Type| MLS ID|       Listing Agent|Status|         Listing URL|
+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+------+--------------------+
|  $164,454|9583 Oak Ave, Fre...|       Fresno|  79339|   IL|   1 bds|     1 ba|  2348 sqft|3615 sqft|      1969|            46|    Apartment|Z535721|Emily Davis - Cen...|  Sold|https://www.zillo...|
|$1,264,926|413 Main St, Fres...|       Fresno|  57401|   CA|   3 bds|     1 ba|  2407 sqft|2609 sqft|      1989|            79|    Apartment|Z282442|Emily Davis - Cen...|  Sold|https://www.zillo...|


In [10]:
df_not_sold = df.where( df.Status != 'Sold')
df_not_sold.show()

+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+
|     Price|             Address|         City|Zipcode|State|Bedrooms|Bathrooms|Area (Sqft)| Lot Size|Year Built|Days on Market|Property Type| MLS ID|       Listing Agent|  Status|         Listing URL|
+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+
|  $554,217|5926 Oak Ave, San...|    San Diego|  65383|   CA|   1 bds|     3 ba|   772 sqft|4757 sqft|      1959|           101|    Townhouse|Z104635|Alex Johnson - Co...|For Sale|https://www.zillo...|
|$1,249,331|8224 Oak Ave, Sac...|   Sacramento|  87393|   TX|   6 bds|     1 ba|  3630 sqft|9369 sqft|      1990|            59|    Townhouse|Z900458|Mike Lee - Coldwe...|For Sale|https://www.

In [14]:
#column string transformations
import pyspark.sql.functions as F
#intermediary columns:
df = df.withColumn('split_bedrooms', F.split('Bedrooms', ' '))
df.show()

+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+--------------+
|     Price|             Address|         City|Zipcode|State|Bedrooms|Bathrooms|Area (Sqft)| Lot Size|Year Built|Days on Market|Property Type| MLS ID|       Listing Agent|  Status|         Listing URL|split_bedrooms|
+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+--------------+
|  $554,217|5926 Oak Ave, San...|    San Diego|  65383|   CA|   1 bds|     3 ba|   772 sqft|4757 sqft|      1959|           101|    Townhouse|Z104635|Alex Johnson - Co...|For Sale|https://www.zillo...|      [1, bds]|
|  $164,454|9583 Oak Ave, Fre...|       Fresno|  79339|   IL|   1 bds|     1 ba|  2348 sqft|3615 sqft|      1969|            46|    

In [15]:
df = df.withColumn('bedroomsINT', df.split_bedrooms.getItem(0).cast(IntegerType()))
df.show()

+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+--------------+-----------+
|     Price|             Address|         City|Zipcode|State|Bedrooms|Bathrooms|Area (Sqft)| Lot Size|Year Built|Days on Market|Property Type| MLS ID|       Listing Agent|  Status|         Listing URL|split_bedrooms|bedroomsINT|
+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+--------------+-----------+
|  $554,217|5926 Oak Ave, San...|    San Diego|  65383|   CA|   1 bds|     3 ba|   772 sqft|4757 sqft|      1959|           101|    Townhouse|Z104635|Alex Johnson - Co...|For Sale|https://www.zillo...|      [1, bds]|          1|
|  $164,454|9583 Oak Ave, Fre...|       Fresno|  79339|   IL|   1 bds|     1 ba|  23

In [16]:
df = df.drop('split_bedrooms')

In [17]:
df = df.withColumn('split_bathrooms', F.split('Bathrooms', ' '))
df = df.withColumn('split_area', F.split('Area (Sqft)', ' '))
df = df.withColumn('split_lot', F.split('Lot Size', ' '))

df.show()

+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+-----------+---------------+------------+------------+
|     Price|             Address|         City|Zipcode|State|Bedrooms|Bathrooms|Area (Sqft)| Lot Size|Year Built|Days on Market|Property Type| MLS ID|       Listing Agent|  Status|         Listing URL|bedroomsINT|split_bathrooms|  split_area|   split_lot|
+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+-----------+---------------+------------+------------+
|  $554,217|5926 Oak Ave, San...|    San Diego|  65383|   CA|   1 bds|     3 ba|   772 sqft|4757 sqft|      1959|           101|    Townhouse|Z104635|Alex Johnson - Co...|For Sale|https://www.zillo...|          1|        [3, ba]| [7

In [18]:
df = df.withColumn('bathroomINT', df.split_bathrooms.getItem(0).cast(IntegerType()))
df = df.withColumn('AreaInt', df.split_area.getItem(0).cast(IntegerType()))
df = df.withColumn('LotINT', df.split_lot.getItem(0).cast(IntegerType()))
df = df.drop('split_bathrooms')
df = df.drop('split_lot')
df = df.drop('split_area')
df.show()

+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+-----------+-----------+-------+------+
|     Price|             Address|         City|Zipcode|State|Bedrooms|Bathrooms|Area (Sqft)| Lot Size|Year Built|Days on Market|Property Type| MLS ID|       Listing Agent|  Status|         Listing URL|bedroomsINT|bathroomINT|AreaInt|LotINT|
+----------+--------------------+-------------+-------+-----+--------+---------+-----------+---------+----------+--------------+-------------+-------+--------------------+--------+--------------------+-----------+-----------+-------+------+
|  $554,217|5926 Oak Ave, San...|    San Diego|  65383|   CA|   1 bds|     3 ba|   772 sqft|4757 sqft|      1959|           101|    Townhouse|Z104635|Alex Johnson - Co...|For Sale|https://www.zillo...|          1|          3|    772|  4757|
|  $164,454|9583 Oak Ave, Fre...|   

In [12]:
#casting to other types:
#df.withColumn('split_bedrooms', df['_c17']).cast(StringType())
#arrayType() column function


DataFrame[summary: string, Price: string, Address: string, City: string, Zipcode: string, State: string, Bedrooms: string, Bathrooms: string, Area (Sqft): string, Lot Size: string, Year Built: string, Days on Market: string, Property Type: string, MLS ID: string, Listing Agent: string, Status: string, Listing URL: string]

In [23]:
#Conditional clouses:

df.select(df.Address, df.City, df.Zipcode, df.State, df['Days on Market'],
    F.when(df['Days on Market'] > 100, "long").otherwise("short").alias("Market_Duration")).show()

+--------------------+-------------+-------+-----+--------------+---------------+
|             Address|         City|Zipcode|State|Days on Market|Market_Duration|
+--------------------+-------------+-------+-----+--------------+---------------+
|5926 Oak Ave, San...|    San Diego|  65383|   CA|           101|           long|
|9583 Oak Ave, Fre...|       Fresno|  79339|   IL|            46|          short|
|8224 Oak Ave, Sac...|   Sacramento|  87393|   TX|            59|          short|
|232 Oak Ave, Fres...|       Fresno|  38666|   TX|           119|           long|
|5446 Pine Rd, Los...|  Los Angeles|  23989|   CA|            26|          short|
|9202 Maple Dr, Sa...|   Sacramento|  71318|   NY|            35|          short|
|2698 Elm St, Sacr...|   Sacramento|  57421|   NY|            94|          short|
|2118 Main St, San...|    San Diego|  64361|   IL|            75|          short|
|9474 Main St, San...|San Francisco|  13227|   FL|            79|          short|
|8803 Maple Dr, 

In [ ]:
# UDF
from pyspark.sql.functions import udf

def reverseString(mystr):
    return mystr[::-1]

udfReverse = udf(reverseString, StringType())

In [25]:
#cashing 
import time

start = time.time()
df = df.distinct().cache()
print("counting %d rows took %f s" % (df.count(), time.time() - start))
start = time.time()
print("Counting %d rows took %f s" % (df.count(), time.time() - start))


counting 3000 rows took 5.087155 s
Counting 3000 rows took 0.999327 s


In [30]:
print(df.is_cached)
df.unpersist()
print(df.is_cached)

False
False


In [ ]:
df.explain() #for getting info about execution plan

In [ ]:
#broadcasting - provides a copy of an object to each worker
# +++ increase join operations!!!
from pyspark.sql.functions import broadcast